In [1]:
%matplotlib notebook

In [62]:
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
import os
import json
from tqdm import tqdm
from collections import defaultdict
import seml
import pandas as pd

In [35]:
collection = seml.database.get_collection('week1_no_model_logging')

In [42]:
results = [{'config' : r['config'], 'result' : r['result']} for r in collection.find()]

In [51]:
results_by_dataset_and_model_type = defaultdict(lambda: defaultdict(list))
for result in results:
    results_by_dataset_and_model_type[result['config']['data']['dataset']][result['config']['model']['model_type']].append(result)

In [67]:
best_scores = defaultdict(dict)
best_configs = defaultdict(dict)
for dataset, dresults in results_by_dataset_and_model_type.items():
    for model_type, mresults in dresults.items():
        best_result = max(mresults, key=lambda result: result['result']['accuracy_mean']['value'])
        score = best_result['result']['accuracy_mean']['value']
        best_scores[dataset][model_type] = score
        best_configs[dataset][model_type] = best_result['config']
        # print(f'\tModel {model_type} ({score * 100:.2f}%) : {best_result["config"]["model"]}')

In [68]:
df = pd.DataFrame(best_scores)
df

,cora_ml,citeseer,pubmed
gat,0.734713,0.702254,0.705408
gcn,0.838254,0.687429,0.849486
gin,0.796866,0.646222,0.815742
sage,0.786794,0.680794,0.834072
mlp,0.651962,0.565206,0.827582
appnp,0.846220,0.699333,0.864565
